In [4]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

In [5]:
df = pd.read_csv('../ROOT_DATA/nue_&_some_mu.csv', header=None, names = ['isTrueTrack', 'pdg', 'completeness', 'purity', 'x', 'z'])

In [6]:
df.head()

,isTrueTrack,pdg,completeness,purity,x,z
0,1,2212,1.000000,0.909091,119.384 119.3 119.172 119.087 119.007 118.93 1...,846.25 846.55 846.85 847.15 847.45 847.75 848....
1,1,2212,0.733333,1.000000,115.512 115.676 115.968 116.284 116.596 116.92...,842.35 842.65 842.95 843.25 843.55 843.85 844....
2,0,11,0.892892,0.998911,165.12 165.08 165.017 164.967 164.921 164.883 ...,640.75 641.05 641.35 641.65 641.95 642.25 642....
3,1,2212,0.989362,0.965398,165.52 165.513 165.514 165.499 165.503 165.496...,638.35 638.65 638.95 639.25 639.55 639.85 640....
4,1,2212,0.875000,0.875000,164.341 164.365 164.386 164.427 164.472 164.52...,632.05 632.35 632.65 632.95 633.25 633.55 633....


In [7]:
for j in range(len(df)):
    print(j)
    df['x'][j] = [float(i) for i in df['x'][j].split()]
    df['z'][j] = [float(i) for i in df['z'][j].split()]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138


3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4172


In [27]:
def RSS(df):
    rssList = []
    for i in range(len(df)):
        perp_dist=0
        length=len(df.z[i])
        reg = LinearRegression().fit(np.array(df.x[i]).reshape(-1,1), df.z[i])
        y_pred = reg.predict(np.array(df.x[i]).reshape(-1,1))
        for j in range(length):
            perp_dist+=abs(-reg.coef_[0]*df.x[i][j] + df.z[i][j] -reg.intercept_)/(reg.coef_[0]**2 + 1)**0.5
        #rssList.append(mean_squared_error(df.z[i], y_pred))
        rssList.append(perp_dist/length)
    return rssList

In [9]:
def RSS2(df):
    rssList = []
    for i in range(len(df)):
        x = np.array(df.x[i]).reshape(-1,1)
        x2 = np.multiply(np.array(df.x[i]), np.array(df.x[i])).reshape(-1,1)
        xx = [[x[i][0], x2[i][0]] for i in range(len(x))]
        reg = LinearRegression().fit(xx, df.z[i])
        y_pred = reg.predict(xx)
        rssList.append(mean_squared_error(df.z[i], y_pred))
    return rssList

In [10]:
def pc2Var(df):
    pc2VarList = []
    scaler = StandardScaler()
    df2 = df.copy(deep=True)
    for i in range(len(df)):
        #print(i)
        df2.x[i] = scaler.fit_transform(np.array(df2.x[i]).reshape(-1,1))
        #df.z[i] = scaler.fit_transform(np.array(df.z[i]).reshape(-1,1))
        data = [[np.array(df2.x[i])[j], np.array(df2.z[i])[j]] for j in range(len(df2.x[i]))]
        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[1])
        print(data.shape)
        pca = PCA(n_components=1)
        pca.fit_transform(data)
        pc2VarList.append(1- pca.explained_variance_ratio_)
    return pc2VarList

In [11]:
pc2Var = pc2Var(df)


(41, 2)
(13, 2)
(247, 2)
(136, 2)
(21, 2)
(317, 2)
(37, 2)
(73, 2)
(242, 2)
(38, 2)
(57, 2)
(71, 2)
(8, 2)
(12, 2)
(164, 2)
(7, 2)
(5, 2)
(41, 2)
(41, 2)
(24, 2)
(281, 2)
(25, 2)
(46, 2)
(485, 2)
(59, 2)
(59, 2)
(116, 2)
(433, 2)
(235, 2)
(56, 2)
(72, 2)
(351, 2)
(63, 2)
(31, 2)
(29, 2)
(398, 2)
(5, 2)
(3, 2)
(171, 2)
(23, 2)
(116, 2)
(251, 2)
(6, 2)
(4, 2)
(1397, 2)
(54, 2)
(33, 2)
(9, 2)
(22, 2)
(73, 2)
(75, 2)
(53, 2)
(1436, 2)
(6, 2)
(580, 2)
(47, 2)
(338, 2)
(35, 2)
(141, 2)
(7, 2)
(169, 2)
(48, 2)
(120, 2)
(28, 2)
(48, 2)
(8, 2)
(860, 2)
(6, 2)
(26, 2)
(6, 2)
(112, 2)
(394, 2)
(9, 2)
(140, 2)
(219, 2)
(10, 2)
(96, 2)
(517, 2)
(16, 2)
(893, 2)
(8, 2)
(157, 2)
(45, 2)
(1593, 2)
(16, 2)
(26, 2)
(4, 2)
(217, 2)
(70, 2)
(58, 2)
(15, 2)
(28, 2)
(8, 2)
(12, 2)
(9, 2)
(645, 2)
(30, 2)
(716, 2)
(125, 2)
(19, 2)
(4, 2)
(7, 2)
(426, 2)
(20, 2)
(5, 2)
(142, 2)
(23, 2)
(9, 2)
(746, 2)
(223, 2)
(456, 2)
(392, 2)
(291, 2)
(21, 2)
(534, 2)
(99, 2)
(22, 2)
(693, 2)
(67, 2)
(16, 2)
(22, 2)
(5, 2)


(337, 2)
(22, 2)
(13, 2)
(3, 2)
(263, 2)
(21, 2)
(31, 2)
(14, 2)
(41, 2)
(611, 2)
(9, 2)
(91, 2)
(8, 2)
(1527, 2)
(37, 2)
(30, 2)
(3, 2)
(3, 2)
(188, 2)
(30, 2)
(49, 2)
(519, 2)
(13, 2)
(4, 2)
(9, 2)
(6, 2)
(563, 2)
(184, 2)
(102, 2)
(28, 2)
(3, 2)
(64, 2)
(20, 2)
(60, 2)
(10, 2)
(894, 2)
(49, 2)
(396, 2)
(162, 2)
(177, 2)
(55, 2)
(140, 2)
(53, 2)
(185, 2)
(218, 2)
(264, 2)
(49, 2)
(24, 2)
(7, 2)
(1658, 2)
(28, 2)
(30, 2)
(122, 2)
(4, 2)
(33, 2)
(68, 2)
(275, 2)
(139, 2)
(118, 2)
(16, 2)
(461, 2)
(7, 2)
(4, 2)
(1265, 2)
(41, 2)
(153, 2)
(84, 2)
(184, 2)
(33, 2)
(50, 2)
(386, 2)
(187, 2)
(26, 2)
(503, 2)
(39, 2)
(7, 2)
(902, 2)
(19, 2)
(4, 2)
(7, 2)
(7, 2)
(206, 2)
(229, 2)
(69, 2)
(11, 2)
(12, 2)
(34, 2)
(16, 2)
(172, 2)
(73, 2)
(25, 2)
(226, 2)
(326, 2)
(87, 2)
(13, 2)
(152, 2)
(22, 2)
(204, 2)
(19, 2)
(81, 2)
(16, 2)
(13, 2)
(17, 2)
(549, 2)
(19, 2)
(921, 2)
(62, 2)
(49, 2)
(26, 2)
(342, 2)
(694, 2)
(258, 2)
(3, 2)
(306, 2)
(65, 2)
(372, 2)
(841, 2)
(73, 2)
(10, 2)
(18, 2)
(8, 2)
(83

(46, 2)
(446, 2)
(302, 2)
(142, 2)
(142, 2)
(9, 2)
(80, 2)
(137, 2)
(316, 2)
(127, 2)
(20, 2)
(637, 2)
(17, 2)
(4, 2)
(86, 2)
(276, 2)
(36, 2)
(863, 2)
(902, 2)
(92, 2)
(13, 2)
(12, 2)
(6, 2)
(43, 2)
(3, 2)
(3, 2)
(348, 2)
(109, 2)
(495, 2)
(44, 2)
(10, 2)
(19, 2)
(54, 2)
(24, 2)
(349, 2)
(7, 2)
(54, 2)
(48, 2)
(73, 2)
(32, 2)
(231, 2)
(70, 2)
(604, 2)
(60, 2)
(67, 2)
(29, 2)
(29, 2)
(9, 2)
(3, 2)
(12, 2)
(6, 2)
(497, 2)
(45, 2)
(61, 2)
(54, 2)
(113, 2)
(75, 2)
(211, 2)
(22, 2)
(52, 2)
(555, 2)
(21, 2)
(444, 2)
(42, 2)
(21, 2)
(36, 2)
(945, 2)
(13, 2)
(47, 2)
(5, 2)
(4, 2)
(26, 2)
(32, 2)
(5, 2)
(169, 2)
(93, 2)
(115, 2)
(98, 2)
(110, 2)
(411, 2)
(11, 2)
(1459, 2)
(196, 2)
(118, 2)
(387, 2)
(64, 2)
(580, 2)
(78, 2)
(371, 2)
(494, 2)
(19, 2)
(19, 2)
(52, 2)
(752, 2)
(8, 2)
(128, 2)
(67, 2)
(15, 2)
(1000, 2)
(208, 2)
(26, 2)
(581, 2)
(843, 2)
(41, 2)
(82, 2)
(141, 2)
(128, 2)
(4, 2)
(322, 2)
(436, 2)
(5, 2)
(36, 2)
(168, 2)
(255, 2)
(70, 2)
(370, 2)
(195, 2)
(41, 2)
(154, 2)
(66, 2)
(137

(155, 2)
(939, 2)
(213, 2)
(22, 2)
(9, 2)
(17, 2)
(77, 2)
(934, 2)
(30, 2)
(176, 2)
(91, 2)
(81, 2)
(97, 2)
(610, 2)
(220, 2)
(80, 2)
(64, 2)
(597, 2)
(9, 2)
(218, 2)
(12, 2)
(49, 2)
(16, 2)
(470, 2)
(8, 2)
(83, 2)
(64, 2)
(113, 2)
(71, 2)
(51, 2)
(12, 2)
(128, 2)
(19, 2)
(830, 2)
(14, 2)
(295, 2)
(7, 2)
(87, 2)
(79, 2)
(13, 2)
(40, 2)
(15, 2)
(42, 2)
(65, 2)
(31, 2)
(45, 2)
(515, 2)
(87, 2)
(23, 2)
(32, 2)
(29, 2)
(35, 2)
(12, 2)
(187, 2)
(1481, 2)
(1085, 2)
(33, 2)
(4, 2)
(146, 2)
(15, 2)
(133, 2)
(899, 2)
(46, 2)
(3, 2)
(3, 2)
(296, 2)
(8, 2)
(117, 2)
(70, 2)
(34, 2)
(947, 2)
(5, 2)
(11, 2)
(9, 2)
(4, 2)
(91, 2)
(353, 2)
(11, 2)
(65, 2)
(65, 2)
(80, 2)
(350, 2)
(19, 2)
(8, 2)
(79, 2)
(8, 2)
(36, 2)
(8, 2)
(856, 2)
(15, 2)
(22, 2)
(183, 2)
(996, 2)
(215, 2)
(39, 2)
(341, 2)
(4, 2)
(39, 2)
(96, 2)
(18, 2)
(20, 2)
(25, 2)
(135, 2)
(226, 2)
(558, 2)
(7, 2)
(361, 2)
(147, 2)
(134, 2)
(709, 2)
(80, 2)
(465, 2)
(445, 2)
(98, 2)
(138, 2)
(578, 2)
(51, 2)
(10, 2)
(3, 2)
(57, 2)
(18, 2)
(8, 2

(98, 2)
(20, 2)
(78, 2)
(58, 2)
(225, 2)
(970, 2)
(731, 2)
(103, 2)
(38, 2)
(38, 2)
(123, 2)
(16, 2)
(60, 2)
(121, 2)
(36, 2)
(30, 2)
(372, 2)
(31, 2)
(496, 2)
(542, 2)
(719, 2)
(89, 2)
(288, 2)
(80, 2)
(4, 2)
(46, 2)
(55, 2)
(497, 2)
(14, 2)
(91, 2)
(63, 2)
(212, 2)
(88, 2)
(1027, 2)
(831, 2)
(925, 2)
(649, 2)
(122, 2)
(47, 2)
(46, 2)
(349, 2)
(22, 2)
(4, 2)
(1468, 2)
(69, 2)
(50, 2)
(96, 2)
(8, 2)
(7, 2)
(390, 2)
(51, 2)
(284, 2)
(90, 2)
(51, 2)
(16, 2)
(30, 2)
(326, 2)
(82, 2)
(31, 2)
(64, 2)
(283, 2)
(172, 2)
(49, 2)
(260, 2)
(62, 2)
(9, 2)
(19, 2)
(394, 2)
(125, 2)
(980, 2)
(97, 2)
(69, 2)
(22, 2)
(23, 2)
(777, 2)
(381, 2)
(315, 2)
(109, 2)
(36, 2)
(126, 2)
(6, 2)
(19, 2)
(15, 2)
(13, 2)
(60, 2)
(69, 2)
(333, 2)
(16, 2)
(11, 2)
(297, 2)
(8, 2)
(224, 2)
(9, 2)
(12, 2)
(22, 2)
(423, 2)
(1179, 2)
(80, 2)
(393, 2)
(179, 2)
(180, 2)
(238, 2)
(39, 2)
(45, 2)
(20, 2)
(88, 2)
(27, 2)
(9, 2)
(25, 2)
(827, 2)
(6, 2)
(79, 2)
(47, 2)
(12, 2)
(1029, 2)
(11, 2)
(885, 2)
(154, 2)
(15, 2)
(10, 2)

In [12]:
pc2Var = [pc2Var[i][0] for i in range(len(pc2Var))]

In [15]:
def findEps(df):
    epsList = []
    for i in range(len(df)):
        print(i)
        epses=[]
        eps = 0.1
        clustering = DBSCAN(eps=eps, min_samples=2).fit(np.array(df['x'][i]).reshape(-1,1), df['z'][i])
        while (list(clustering.labels_).count(0)/len(clustering.labels_)< 1):
            eps+=0.1
            if eps > 20:
                eps += 1
            if eps> 100:
                break

            clustering = DBSCAN(eps=eps, min_samples=2).fit(np.array(df['x'][i]).reshape(-1,1), df['z'][i])

        epsList.append(eps)
    #print(eps)
    return epsList

In [16]:
epsList = findEps(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068


3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707


In [28]:
RSS = RSS(df)
RSS2 = RSS2(df)

In [18]:
epsList


[0.2,
 0.5,
 4.899999999999999,
 0.7,
 0.1,
 4.799999999999999,
 0.6,
 14.999999999999963,
 1.8000000000000005,
 0.7,
 12.79999999999997,
 21.000000000000014,
 0.4,
 0.2,
 18.9,
 0.1,
 0.30000000000000004,
 0.2,
 6.499999999999993,
 0.1,
 1.6000000000000003,
 0.30000000000000004,
 0.8999999999999999,
 9.299999999999983,
 0.1,
 5.699999999999996,
 22.100000000000016,
 14.799999999999963,
 0.8999999999999999,
 2.400000000000001,
 1.3,
 0.5,
 0.7,
 1.6000000000000003,
 0.9999999999999999,
 7.099999999999991,
 0.2,
 0.5,
 13.09999999999997,
 0.7999999999999999,
 3.1000000000000014,
 1.4000000000000001,
 0.4,
 0.1,
 3.600000000000002,
 2.2000000000000006,
 0.2,
 0.6,
 0.8999999999999999,
 9.399999999999983,
 0.1,
 0.4,
 22.100000000000016,
 0.1,
 10.499999999999979,
 1.3,
 8.799999999999985,
 0.4,
 1.0999999999999999,
 0.1,
 1.7000000000000004,
 0.9999999999999999,
 1.0999999999999999,
 0.1,
 0.30000000000000004,
 0.30000000000000004,
 5.799999999999995,
 4.699999999999999,
 0.6,
 7.9999999

In [19]:
nHits = [len(df.x[i]) for i in range(len(df)) ]

In [173]:
def slidingWindowHelper(x,y, n):
    
    reg=LinearRegression().fit(x.reshape(-1,1), y)
    reg.coef_, reg.intercept_
    d=0.5
    m = reg.coef_[0]
    
    
    if m == 0:
        return 0

    dx= d/(m*m +1)**0.5 
    
    def y_pred(x):
        return float(m*x+reg.intercept_)
    
    intercepts = [(1/float(m))*i + y_pred(i) for i in np.arange(min(x),max(x),float(dx)) ]
    #print(len(intercepts), m)
    perp_dist=0
    #plt.scatter(x, y)
    #plt.show()

    for i in range(0,len(intercepts)-1):
        bighits=[]
        smallhits=[]
        
        if m > 0:
            for j in range(len(x)):
                if y[j]<-1/m*x[j] + intercepts[min(i+n+1,len(intercepts)-1)] and y[j]>-1/m*x[j] + intercepts[max(i-n,0)]:
                    bighits.append(j)
                if y[j]<-1/m*x[j] + intercepts[min(i+1,len(intercepts)-1)] and y[j]>-1/m*x[j] + intercepts[i]:
                    smallhits.append(j)
        else:
            for j in range(len(x)):
                if y[j]>-1/m*x[j] + intercepts[min(i+n+1,len(intercepts)-1)] and y[j]<-1/m*x[j] + intercepts[max(i-n,0)]:
                    bighits.append(j)
                if y[j]>-1/m*x[j] + intercepts[min(i+1,len(intercepts)-1)] and y[j]<-1/m*x[j] + intercepts[i]:
                    smallhits.append(j)
            
        #print(smallhits)
        if len(smallhits)==0:
            continue
        new_x = np.array([x[k] for k in bighits])
        new_y = [y[k] for k in bighits]
        
        new_reg=LinearRegression().fit(new_x.reshape(-1,1), new_y)
        for l in smallhits:
            perp_dist+=abs(-new_reg.coef_[0]*x[l] + y[l] -new_reg.intercept_)/(new_reg.coef_[0]**2 + 1)**0.5
    return perp_dist/len(x)      

In [174]:
def SlidingWindow(df,n):
    slidingWindowList=[]
    for i in range(len(df)):

        #print(df.x[i], df.z[i])
        answer = slidingWindowHelper(np.array(df.x[i]),np.array(df.z[i]), n )
        print(i,answer)
        slidingWindowList.append(answer)
    return slidingWindowList

In [175]:
sliding = SlidingWindow(df,4)
sliding

0 0.0102792629737
1 0.0163468607748
2 0.22483278905
3 0.00610794102814
4 0.00866950130443
5 0.214042400429
6 0.0560910030779
7 0.387845088774
8 0.0134297525293
9 0.0230828321644
10 0.0553706369455
11 0.0576458769875
12 0.0171414639093
13 0.0235897866796
14 0.453690497871
15 0.000477488269669
16 0.0
17 0.0469474016672
18 0.0991971214653
19 0.000553987570042
20 0.0128239935444
21 0.0330928632653
22 0.19977178482
23 0.389938969858
24 0.00641015039972
25 0.0143247123412
26 0.00843940030128
27 0.374136427909
28 0.252114936292
29 0.520554488563
30 0.00569897674021
31 0.0109421493355
32 0.0411120914007
33 0.193978527003
34 0.0693355334437
35 0.335148467009
36 0.0048243693135
37 0.0
38 0.362587072806
39 0.0342681422832
40 0.438549317332
41 0.268050181293
42 0.0134168640237
43 0.0
44 0.313876367607
45 1.53140004028
46 0.0436944341049
47 0.0498171347452
48 0.0122040241313
49 0.0287825316218
50 0.0120040726925
51 0.0731737852316
52 0.265991656519
53 0.00191992137154
54 0.539529139147
55 0.0124224

445 0.00679643148814
446 0.0394435524667
447 0.82148703678
448 0.0174079498905
449 0.573783245457
450 0.408657610162
451 0.0
452 0.297344562238
453 0.011699904976
454 0.269144186769
455 0.0
456 0.391653653434
457 0.00399925409183
458 0.0458606176562
459 0.34424304309
460 0.00446475978383
461 0.00545115275442
462 0.0127137915421
463 0.433625919766
464 0.0431710407163
465 0.00869019891861
466 0.0934652845348
467 0.0620811829188
468 0.343269424888
469 0.016222432448
470 0.0929484896177
471 0.313896979488
472 0.0521444044841
473 0.0067578559014
474 0.220748232026
475 0.381891144989
476 0.219265978185
477 0.0
478 0.293594227278
479 0.288661016469
480 0.316162462968
481 0.00118660708787
482 0.0702363234218
483 0.00682113826818
484 0.139844024926
485 0.017719602519
486 0.623290168867
487 0.336576654949
488 0.0927145860039
489 0.0085604888661
490 0.190948020179
491 0.0
492 0.224069819222
493 0.297785100088
494 0.0421166496426
495 0.368134564277
496 0.0
497 0.00824171476385
498 0.0524679892931


883 0.333206193362
884 0.00593069648887
885 0.442787704601
886 0.0149375766919
887 0.009697570854
888 0.0280138949182
889 0.32558203996
890 0.00979547842459
891 0.400048403144
892 0.164757451929
893 0.354495697312
894 0.281234863107
895 0.120344868001
896 0.00844241496314
897 0.213323751429
898 0.148866144899
899 0.0332649222185
900 0.010708979424
901 0.1285617321
902 0.0299369568335
903 0.298219684807
904 0.0753686832735
905 0.310469787267
906 0.00990309163329
907 0.0591071463652
908 0.504784543266
909 0.0249061551087
910 0.0158564735074
911 0.30531766865
912 0.297282492293
913 0.217126140995
914 0.265310612709
915 0.367879827227
916 0.0219476852359
917 0.0834548162958
918 0.389922096686
919 0.0023794399096
920 0.547071973658
921 0.00863483686723
922 0.665410778783
923 0.234413871484
924 0.26654133691
925 0.192405886768
926 0.00426286855519
927 0.0258107922975
928 0.0773114110029
929 0.451305747193
930 0.0515267885181
931 0.486052127118
932 0.00869615996344
933 0.285899267408
934 0.01

1298 0.378250826659
1299 0.313396943677
1300 0.185527144582
1301 0.0
1302 0.0159764108856
1303 0.016936988816
1304 0.0409413078528
1305 0.320446847538
1306 0.273078309791
1307 0.30114538787
1308 0.397850257098
1309 0.0149735196919
1310 0.00512438285216
1311 0.272437282988
1312 0.00893899327547
1313 0.0702670550568
1314 0.274847399907
1315 0.198107065501
1316 0.122878462306
1317 0.0190296612564
1318 0.319312137434
1319 0.261405023152
1320 0.0227847839876
1321 0.445257525955
1322 0.00776385082229
1323 0.497126733687
1324 0.00688309570935
1325 0.00901408071036
1326 0.32180620505
1327 0.437621440231
1328 0.10462952054
1329 0.503685704089
1330 0.392744713644
1331 0.114535288395
1332 0.149051964185
1333 0.00732134856861
1334 0.0297785054557
1335 0.0505633119283
1336 0.383607930691
1337 0.514927506334
1338 0.00792108602955
1339 0.028293047145
1340 0.437984506043
1341 0.296697812149
1342 0.0378824951635
1343 0.0212627939225
1344 0.00581160276495
1345 0.269341081365
1346 0.350450361759
1347 0.0

1706 0.294333637084
1707 0.347387234721
1708 0.0167169550557
1709 0.348316158214
1710 0.0133117990527
1711 0.299426878882
1712 0.578825565893
1713 0.0300092279489
1714 0.229906115788
1715 0.255678687329
1716 0.0117758595769
1717 0.426216586315
1718 0.0102186566503
1719 0.157105285769
1720 0.47397652808
1721 0.0240894408298
1722 0.184103501221
1723 0.367630545929
1724 0.606426394223
1725 0.0111357830124
1726 0.00972445075948
1727 0.107357536912
1728 0.0359449344808
1729 0.0124407586283
1730 0.2658334577
1731 0.2413732113
1732 0.0139931671966
1733 0.0115125768663
1734 0.29222305231
1735 0.150599104209
1736 0.020718221272
1737 0.361696247689
1738 0.278016997662
1739 0.274137578393
1740 0.0188368018979
1741 0.00187153973338
1742 0.014615611588
1743 0.149740236
1744 0.605358357265
1745 0.103559559298
1746 0.0
1747 0.459857870763
1748 0.141445665644
1749 0.0162571694282
1750 0.238702523256
1751 0.0517250724463
1752 0.366244874
1753 0.132262149179
1754 0.0105145044893
1755 0.0
1756 0.43914180

...

3328 0.0108174791304
3329 5.08422994585e-14
3330 0.0
3331 0.0107448750703
3332 0.0297592788673
3333 0.162981455056
3334 0.420444366007
3335 0.375916974665
3336 0.0793627558638
3337 0.237802833789
3338 0.399874057809
3339 0.280496382801
3340 0.00424355913495
3341 0.017560605703
3342 0.0162894926946
3343 0.0110356307239
3344 0.0125329780463
3345 0.0131355316051
3346 0.00743551994295
3347 0.00847627338352
3348 0.0303783247679
3349 0.107128077967
3350 0.0224192470928
3351 0.0171698447914
3352 0.00155074826522
3353 0.0128377973405
3354 0.0173640525011
3355 0.214000626865
3356 0.0156983334824
3357 0.0296911651863
3358 1.74234753903e-14
3359 0.0146173176556
3360 0.032805103645
3361 0.0256868769504
3362 0.00991225720967
3363 0.00593729555168
3364 0.0171837252613
3365 0.00508517922159
3366 0.0100869206819
3367 0.0150219209306
3368 0.00724097864141
3369 0.0479979964916
3370 0.284759850488
3371 0.0126191033158
3372 0.0136192722599
3373 0.0412403642307
3374 0.0248425333409
3375 0.0122955311381
337

3727 0.0185412990326
3728 0.0110415729227
3729 0.0295434888001
3730 0.0106246190142
3731 0.0414767983296
3732 0.00690498317593
3733 2.28018876869e-14
3734 0.00663013676131
3735 0.345094515924
3736 0.00878589536494
3737 0.0190594525747
3738 0.624516055241
3739 0.164754859455
3740 0.0204603171055
3741 0.0066314332081
3742 0.338260382168
3743 0.0309138950783
3744 0.0209908962002
3745 0.0217646963421
3746 0.0146756300423
3747 0.0174932754173
3748 0.0350145379606
3749 0.00372996940768
3750 0.0147290636876
3751 4.85053062111e-15
3752 0.0259519971515
3753 0.289485587782
3754 0.720114060509
3755 0.0283023003688
3756 0.00863792170978
3757 0.0169227760814
3758 0.0152711714241
3759 0.0
3760 0.247009596449
3761 0.0107781079051
3762 0.0143563021398
3763 0.0130266440204
3764 0.251624455816
3765 0.0706770639221
3766 0.0262832894659
3767 0.0255934408312
3768 0.108279966373
3769 0.0170872082579
3770 0.0164636991306
3771 0.376461873074
3772 0.61574222453
3773 0.0113688850088
3774 0.0278741803083
3775 0.

4133 0.349058683446
4134 0.0
4135 0.0103488832134
4136 0.00589425518277
4137 0.0174935863024
4138 0.0301131589371
4139 0.0082125723782
4140 0.515431238495
4141 0.0124959447916
4142 0.254873333889
4143 0.000590190632606
4144 0.0241837849361
4145 0.323758925257
4146 0.202821426887
4147 0.0384369724623
4148 0.0113878745309
4149 0.0243945512057
4150 0.00589953155309
4151 0.00917316172642
4152 0.00615867421173
4153 0.0292366560669
4154 0.0576093800283
4155 0.0014971297588
4156 0.0631200659557
4157 0.00561124655467
4158 0.0140165237019
4159 0.0220591857373
4160 0.0
4161 0.0163599366403
4162 0.0654493628944
4163 0.0318663974732
4164 0.0029283938087
4165 0.430543399709
4166 0.0853523701448
4167 0.0214800849679
4168 0.0261252329221
4169 0.013136750109
4170 0.074090240148
4171 0.0342816680244
4172 0.0160155182102
4173 0.331842967945
4174 0.0943742851678
4175 0.0910421544282
4176 0.00581605337055
4177 0.0123419480164
4178 0.00901079593565
4179 0.136649956221
4180 0.0117791036608
4181 0.0047746655

4532 0.0
4533 0.0154069518287
4534 0.0166736016239
4535 0.0138817209146
4536 0.0516500996902
4537 0.595259773983
4538 0.0220764414661
4539 0.128761618908
4540 0.00777050543368
4541 0.016348450707
4542 0.364197233638
4543 0.0199750824204
4544 0.0180599702042
4545 0.00653974375968
4546 0.0140990440522
4547 0.293945955988
4548 0.0152326375214
4549 0.0127576410065
4550 0.0259433550482
4551 0.0116274383717
4552 0.0349176195197
4553 0.0170248654476
4554 0.0102211820064
4555 0.00095724813691
4556 0.118286081409
4557 0.0999205360829
4558 0.257810857281
4559 0.00898061275603
4560 0.00637666804515
4561 0.259204888516
4562 0.0616252506577
4563 0.0117254232824
4564 0.00863875076995
4565 0.0254260768042
4566 0.0222978147501
4567 0.0114513272183
4568 0.012905654681
4569 0.326824082366
4570 0.442483436588
4571 0.0151878689486
4572 0.00838111597802
4573 0.0135363873148
4574 0.0447739847893
4575 0.0100947265679
4576 0.00822914486305
4577 0.0148139461176
4578 0.016460325564


[0.010279262973714969,
 0.016346860774778257,
 0.22483278904971657,
 0.0061079410281428192,
 0.0086695013044347045,
 0.21404240042865036,
 0.056091003077893942,
 0.38784508877428764,
 0.013429752529302001,
 0.023082832164382245,
 0.055370636945461481,
 0.057645876987531083,
 0.017141463909302615,
 0.02358978667958005,
 0.45369049787050758,
 0.00047748826966862351,
 0.0,
 0.046947401667228723,
 0.099197121465288529,
 0.00055398757004242931,
 0.012823993544395269,
 0.033092863265290803,
 0.19977178482035363,
 0.38993896985835208,
 0.0064101503997185194,
 0.014324712341208932,
 0.0084394003012814127,
 0.37413642790932572,
 0.25211493629195214,
 0.52055448856292474,
 0.0056989767402136201,
 0.010942149335515161,
 0.041112091400672901,
 0.19397852700321,
 0.069335533443744476,
 0.33514846700870243,
 0.0048243693134958398,
 0.0,
 0.36258707280558705,
 0.034268142283227443,
 0.43854931733208735,
 0.26805018129285824,
 0.013416864023708799,
 0.0,
 0.31387636760653109,
 1.531400040280426,
 0.04

sl

In [176]:
df = pd.read_csv('../BNB_nue data/nue_and_mu.csv', index_col=0)
df.head()

,isTrueTrack,pdg,completeness,purity,x,z,eps,pc2Var,nHits,RSS,RSS2
0,1,2212,1.000000,0.909091,"[119.384, 119.3, 119.172, 119.087, 119.007, 11...","[846.25, 846.55, 846.85, 847.15, 847.45, 847.7...",0.2,0.000057,41,0.020997,0.007951
1,1,2212,0.733333,1.000000,"[115.512, 115.676, 115.968, 116.284, 116.596, ...","[842.35, 842.65, 842.95, 843.25, 843.55, 843.8...",0.5,0.000663,13,0.032420,0.002150
2,0,11,0.892892,0.998911,"[165.12, 165.08, 165.017, 164.967, 164.921, 16...","[640.75, 641.05, 641.35, 641.65, 641.95, 642.2...",4.9,0.000467,247,1.072781,47.833154
3,1,2212,0.989362,0.965398,"[165.52, 165.513, 165.514, 165.499, 165.503, 1...","[638.35, 638.65, 638.95, 639.25, 639.55, 639.8...",0.7,0.000035,136,0.130221,0.216665
4,1,2212,0.875000,0.875000,"[164.341, 164.365, 164.386, 164.427, 164.472, ...","[632.05, 632.35, 632.65, 632.95, 633.25, 633.5...",0.1,0.001649,21,0.028268,0.008956


In [179]:
df['sliding']=sliding

,isTrueTrack,pdg,completeness,purity,x,z,eps,pc2Var,nHits,RSS,RSS2,sliding
0,1,2212,1.000000,0.909091,"[119.384, 119.3, 119.172, 119.087, 119.007, 11...","[846.25, 846.55, 846.85, 847.15, 847.45, 847.7...",0.2,0.000057,41,0.020997,0.007951,0.010279
1,1,2212,0.733333,1.000000,"[115.512, 115.676, 115.968, 116.284, 116.596, ...","[842.35, 842.65, 842.95, 843.25, 843.55, 843.8...",0.5,0.000663,13,0.032420,0.002150,0.016347
2,0,11,0.892892,0.998911,"[165.12, 165.08, 165.017, 164.967, 164.921, 16...","[640.75, 641.05, 641.35, 641.65, 641.95, 642.2...",4.9,0.000467,247,1.072781,47.833154,0.224833
3,1,2212,0.989362,0.965398,"[165.52, 165.513, 165.514, 165.499, 165.503, 1...","[638.35, 638.65, 638.95, 639.25, 639.55, 639.8...",0.7,0.000035,136,0.130221,0.216665,0.006108
4,1,2212,0.875000,0.875000,"[164.341, 164.365, 164.386, 164.427, 164.472, ...","[632.05, 632.35, 632.65, 632.95, 633.25, 633.5...",0.1,0.001649,21,0.028268,0.008956,0.008670


In [180]:
df.to_csv('../BNB_nue data/nue_and_mu.csv')

In [29]:
df['eps'] =epsList
df['pc2Var'] = pc2Var
df['nHits']=nHits
df['RSS']=RSS
df['RSS2']=RSS2

In [30]:
df.to_csv('../BNB_nue data/nue_and_mu.csv')

TypeError: 'NoneType' object does not support item assignment

[0.8608684755910746,
 0.42145664798067134,
 264.97700888645892,
 17.710110251206164,
 0.59362196364274245,
 3192.8521345202662,
 83.940630157235432,
 74.599791033737574,
 45.838267239046097,
 2.5579717343445365,
 29.606838998281983,
 52.568790514549427,
 3.0063731062919246,
 0.59115013723342014,
 198.15320272885174,
 0.1114798707459056,
 0.94904771701196711,
 2.3099302384758604,
 18.584934203676319,
 33.637906610772809,
 133.53053631571549,
 1.5736715095585763,
 10.7401852322108,
 1324.9491816502118,
 0.88548040657398608,
 17.811279966187577,
 123.65792004240943,
 6025.3292491384991,
 540.06073556671367,
 40.530799232271818,
 1.3748556723615624,
 178.13257331792988,
 5.5956463183258096,
 8.6652031559356359,
 7.1162646492534805,
 2103.6720177702014,
 0.15729318260060507,
 0.11984016222236793,
 116.53389295943637,
 0.98264077713517428,
 238.49278311006054,
 179.63308758791277,
 0.468886221867238,
 0.063023343473851953,
 4099.4956002876324,
 99.158829885224534,
 44.603479694052126,
 0.979